In [1]:
import imaplib, email, getpass, csv, time
from email.utils import getaddresses
from collections import defaultdict
from time import strftime, gmtime

In [ ]:

#login
imap_server = 'imap.gmail.com'
imap_user = 'jessicasliang@gmail.com'
imap_password = getpass.getpass()

my_emails = {'jessicasliang@gmail.com': True, 'jessica_liang@brown.edu': True, 'jsl15@cs.brown.edu': True}

conn = imaplib.IMAP4_SSL(imap_server)
(retcode, capabilities) = conn.login(imap_user, imap_password)

if retcode != 'OK':
    print "something went wrong with logging in"
else:
    print "login successfulllllllll"

In [3]:
# fetch the folder
conn.select("[Gmail]/All Mail", readonly=True)

result, data = conn.uid('search', None, '(SINCE "01-SEP-2015")')
uids = data[0].split()
print len(uids)
result, data = conn.uid('fetch', ','.join(uids),'(RFC822)')
print result

10986
OK


In [6]:
import re
def parse_payload(payload):
    return None
def parse_address(email_address):
    p = re.compile('\<.*>')
    matches = p.findall(email_address)
    if len(matches) == 0:
        email = email_address
    else: 
        email = matches[0]
        email = email.replace("<", "")
        email = email.replace(">", "")
    return email.lower()

def parse_email(data_entry):
    e = {}
    msg = email.message_from_string(data_entry)
    e['mid'] = msg.get_all('message-id', "")
    e['date'] = time.mktime(email.utils.parsedate(msg.get_all('date', None)[0]))
    e['sender'] = parse_address(msg.get_all('from', [])[0])
    e['receivers'] = msg.get_all('to', None)
    e['subject'] = msg.get_all('subject', "")
    cc = msg.get_all('cc')
    if cc == None or len(cc) == 0:
        e['cc'] = []
    else:
        e['cc'] = cc[0].split(",")
    e['reply-to'] = msg.get_all('in-reply-to', [])
    payload = msg.get_payload()
    e['text'] = parse_payload(payload)
    return e
    

In [9]:
#f = open('emails.tsv', 'w')
all_emails = {}
#fieldnames = ['mids', 'date', 'sender', 'receivers', 'subject', 'cc', 'reply-to','text']
#writer = csv.DictWriter(f, fieldnames=fieldnames, delimiter='\t')
#writer.writeheader()
for i in range(len(data)):
    if len(data[i]) != 2:
        continue
    msg = email.message_from_string(data[i][1])
    if msg.get_all('date', None) == None:
        continue
    row = parse_email(data[i][1])
    all_emails[row['mid'][0]] = row
    #writer.writerow(row)
    
#f.close()

In [13]:
print len(all_emails.keys())

10974


In [16]:
#filter out sent emails
email_by_sender = defaultdict(list)
sent = []
received = []
for mid in all_emails:
    email_by_sender[all_emails[mid]['sender']].append(mid)
    if all_emails[mid]['sender'] in my_emails:
        sent.append(mid)
    else:
        received.append(mid)
        
response_times = {}
sender_rt_total = defaultdict(float)
sender_num_res = defaultdict(int)

# calculate num responses per sender and total response time

for mid in sent:
    sent_email = all_emails[mid]
    reply_ids = sent_email['reply-to']
    # if this is a reply
    if len(reply_ids) > 0:
        reply_email_id = reply_ids[0]
        if reply_email_id in all_emails:
            reply_email = all_emails[reply_email_id]
            rt = sent_email['date'] - reply_email['date']
            sender = reply_email['sender']
            if rt > 0:
                response_times[reply_email_id] = rt
                sender_rt_total[sender] += rt
                sender_num_res[sender] += 1
                
sender_avg_rt = {}
total_responses = 0
total_response_time = 0
for sender in sender_rt_total:
    sender_avg_rt[sender] = sender_rt_total[sender] / float(sender_num_res[sender])
    total_responses += sender_num_res[sender]
    total_response_time += sender_rt_total[sender]

overall_avg = float(total_response_time) / float(total_responses)
print overall_avg
print len(sender_avg_rt.keys())
    
def willRespond(e):
    return not(num_responses[e['sender']] == 0 and total_response_times[e['sender']] >= 5)

#valid_emails = filter(lambda x: willRespond(emails[x]), emails.keys())

139763.467532
117


In [20]:
f = open('sender_avg_rt.tsv', 'w')
fieldnames = ['sender', 'avg', 'count']
writer = csv.DictWriter(f, fieldnames=fieldnames, delimiter='\t')
writer.writeheader()
for key in email_by_sender:
    row ={}
    row['sender'] = key
    if key in sender_avg_rt:
        row['avg'] = sender_avg_rt[key]
    else:
        row['avg'] = -1
    row['count'] = len(email_by_sender[key])
    writer.writerow(row)
f.close()

jonathan_rubins@brown.edu

kristen_mclean@brown.edu
kevin.bishop@marines.usmc.mil
honorata.zaklicki@huffingtonpost.com
ucs@brown.edu
admin@tenxlist.com
wesley_miller@brown.edu
update@em.facebookmail.com
ethomas@tripadvisor.com
awstlaur@cs.brown.edu (alexander st. laurent)
renewals@namecheap.com
res_life@brown.edu
samuel_dooman@brown.edu
infinity@e.disneyinteractive.com
fidelity.alerts@fidelity.com
pvdaradmin@ehi.com
info@ubercollege04.info
miriamk@fb.com
brownusjp@gmail.com
team@sharelatex.com
levinotik@gmail.com
paxton@cs.montana.edu
cody_mello@alumni.brown.edu
young-rae_kim@brown.edu
amanda.merriweather@delphix.com
talha_ismail@brown.edu
d+mta2mtmxnzm2nzi2ndm4otu3mdi5-mte1njq0ntu3mzm4odmzmjg0mdu1@docs.google.com
callie_parker@brown.edu
philip_strauss@brown.edu
aaron_rosenthal@brown.edu
noreply@results.equinox.com
dse_na2@docusign.net
samuel_brebner@brown.edu
messenger@mangafox.com
hello@startupatbrown.org
wilson_cusack@brown.edu
noreply@kimpton-email.com
no-reply@piazzacareers.com
co

In [ ]:
from sklearn import linear_model
from sklearn import cross_validation

# filters
# primary inbox
# I've never responsded to them and they've sent me at least 5 emails
# from the last year
# if you never respond to listervs

def get_features(email_id):
    e = all_emails[email_id]
    features = []
    #features.append(last_sent_time[email_id])
    if e['sender'] in sender_avg_rt:
        features.append(sender_avg_rt[e['sender']])
    else:
        features.append(overall_avg)
    t = time.localtime(e['date'])
    features.append(t.tm_hour)
    features.append(t.tm_wday)
    features.append(len(e['cc']))
    return features
keys = response_times.keys()
featurized = map(get_features, keys)
y = map(lambda x: response_times[x], keys)

regr = linear_model.Lasso()
#print cross_validation.cross_val_score(regr, featurized, y, cv=3)
regr.fit(featurized, y)


# features
# time of day
# day of week
# listerv response rate
# replied to email in the last 15 minutes
# length of email
# average response time to sender
# number of people in message




In [ ]:
import math
print regr.score(featurized, y)
print regr.coef_
f = open('output.txt', 'w')
fieldnames = [ 'real', 'predict', 'diff']
writer = csv.DictWriter(f, fieldnames=fieldnames, delimiter='\t')
writer.writeheader()
keys = response_times.keys()
total_diff = 0
for i in range(len(keys)):
    row = {}
   # row['mid'] = keys[i]
    row['predict'] = regr.predict(get_features(keys[i]))
    row['real'] = response_times[keys[i]]
    row['diff'] = (row['predict'] - row['real'])/60/60
    total_diff += abs(row['diff'])
    writer.writerow(row)

print total_diff/len(keys)
f.close()

In [ ]:
from sklearn.externals import joblib

joblib.dump(regr, 'email_response.pkl')
#clf = joblib.load('email_response.pkl')
